Rep count for knee bend exercise

In [ ]:
#Installing mediapipe and opencv

%pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
#calculating angle between the three points - knee, hip and ankle

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
import time
from time import process_time

#VIDEO FEED
cap = cv2.VideoCapture(os.path.join('KneeBend.mp4'))

#video properties
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)


#creating new video file for videowriter in mp4 codec format
video_writer = cv2.VideoWriter(os.path.join("output.mp4"), cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height)) 

# Curl counter variables
counter = 0     # Rep counter
stage = None    # Knee position - Up or down 
bent = ' '      # feedback string
ett = 0         # end timer
stt = 0         # start timer
timetaken = 0   # holding time taken (in seconds)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    # loop through each frame in video
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
        
            if angle < 120 and stage =='down':
                stage="up"
                bent = " "
                timetaken = 0 
                stt = process_time()
                counter +=1
                print("count",counter)

            if angle > 160 and stage=="up":
                stage = "down"
                ett = process_time()
                timetaken = ett-stt
                print("time taken" ,int(ett-stt))
                if (int(ett-stt))<8:
                    bent = "Keep your Knee bent"
                    print(bent)

            if angle > 160:
                stage = "down"
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (1000,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (75,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (75,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)

        # timer
        cv2.putText(image, 'TIME TAKEN', (150,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(int(timetaken)), 
                    (180,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)

        # feedback
        cv2.putText(image, 'FEEDBACK', (270,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, bent, 
                    (270,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
        
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        # show the results
        cv2.imshow('Mediapipe Feed', image)

        # write each frame 
        video_writer.write(image)

        # break the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release cap
    cap.release()
    cv2.destroyAllWindows()

    # Release videowriter
    video_writer.release()